# GPS Parsers

```
$GPGGA,234804.000,3906.7106,N,12120.3144,W,1,04,1.77,135.6,M,-22.1,M,,*51
$GPGSA,A,3,03,32,02,04,,,,,,,,,2.01,1.77,0.95*08
$GPRMC,234804.000,A,3906.7106,N,12120.3144,W,0.10,107.17,120823,,,A*70
$GPVTG,107.17,T,,M,0.10,N,0.19,K,A*34
$GPGGA,234805.000,3906.7106,N,12120.3144,W,1,04,1.77,135.6,M,-22.1,M,,*50
$GPGSA,A,3,03,32,02,04,,,,,,,,,2.01,1.77,0.95*08
$GPRMC,234805.000,A,3906.7106,N,12120.3144,W,0.17,95.16,120823,,,A*4D
$GPVTG,95.16,T,,M,0.17,N,0.31,K,A*02
```

Message | Function
--------|------------
DP  | Dynamic positioning
DTM	| Datum reference information
GBS	| GNSS satellite fault detection (RAIM support)
[GGA][1]	| Time, position, and fix related data
[GLL][5]	| Position data: position fix, time of position fix, and status
GNS	| GNS Fix data
GRS	| GRS range residuals
[GSA][6]	| GPS DOP and active satellites
GST	| Position error statistics
[GSV][3]	| Number of SVs in view, PRN, elevation, azimuth, and SNR
HDT	| Heading from True North
[RMC][2]	| Position, Velocity, and Time
[VTG][7]	| Actual track made good and speed over ground
[ZDA][4]	| UTC day, month, and year, and local time zone offset


[1]: https://www.ae.utexas.edu/courses/ase389p7/projects/svatek/commands/GGA.html
[2]: https://www.ae.utexas.edu/courses/ase389p7/projects/svatek/commands/RMC.html
[3]: https://www.ae.utexas.edu/courses/ase389p7/projects/svatek/commands/GSV.html
[4]: https://www.ae.utexas.edu/courses/ase389p7/projects/svatek/commands/ZDA.html
[5]: https://www.ae.utexas.edu/courses/ase389p7/projects/svatek/commands/GLL.html
[6]: https://www.ae.utexas.edu/courses/ase389p7/projects/svatek/commands/GSA.html
[7]: https://www.ae.utexas.edu/courses/ase389p7/projects/svatek/commands/VTG.html

## References

- [GPS message format](https://gpsd.gitlab.io/gpsd/NMEA.html#_gga_global_positioning_system_fix_data)
- [NEMA messages](https://receiverhelp.trimble.com/alloy-gnss/en-us/NMEA-0183messages_MessageOverview.html)

In [7]:
from serial import Serial
from serial.tools.list_ports import comports, grep
from collections import namedtuple

In [2]:
GSA = namedtuple("GSA", "id mode fix prns pdop hdop vdop")
RMC = namedtuple("RMC", "id utc status lat lon speed track data mag")
GGA = namedtuple("GGA", "id utc lat lon qual num_sats hdop msl geoid age ref_id")
time_t = namedtuple("time_t", "hr min sec")
date_t = namedtuple("data_t", "day month year")
sat_t = namedtuple("sat_t", "prn el az snr")

In [3]:

def parseHex(c):
  if (c < '0'):
    return 0
  if (c <= '9'):
    return ord(c) - ord('0')
  if (c < 'A'):
    return 0
  if (c <= 'F'):
    return ord(c) - ord('A') + 10
  return 0

def valid_msg(msg):
    cs = 0
    for c in msg[1:-3]:
        cs = cs ^ ord(c)

    cs = cs & 0xff
    cc = parseHex(msg[-2])*16 + parseHex(msg[-1])
    return cc == cs

def gsa_parser(msg):
    """
    $GPGSA,A,3,03,32,02,04,,,,,,,,,2.01,1.77,0.95*08
    """
    chunks = msg[1:-3].split(',')
    match int(chunks[2]):
        case 1: fix = "unavailable"
        case 2: fix = "2D"
        case 3: fix = "3D"
        case _: fix = chunks[2]

    msg = GSA(
        chunks[0],
        chunks[1],
        fix,
        chunks[3:15],
        float(chunks[15]),
        float(chunks[16]),
        float(chunks[17])
    )
    return msg

def rmc_parser(msg):
    """
    $GPRMC,234805.000,A,3906.7106,N,12120.3144,W,0.17,95.16,120823,,,A*4D
    """
    chunks = msg[1:-3].split(',')
    id = chunks[0]
    utc = time_t(
        int(chunks[1][:2]),
        int(chunks[1][2:4]),
        float(chunks[1][4:])
    )
    lat = float(chunks[3][:2]) + float(chunks[3][2:]) / 60.0
    if chunks[4] == 'S':
        lat = -lat
    lon = float(chunks[5][:3]) + float(chunks[5][3:]) / 60.0
    if chunks[6] == 'W':
        lon = -lon

    msg = RMC(
        id,
        utc,
        chunks[2],
        lat,
        lon,
        float(chunks[7]),
        float(chunks[8]),
        date_t(
            int(chunks[9][:2]),
            int(chunks[9][2:4]),
            int(chunks[9][4:])
        ),
        "" if chunks[10] == "" else float(chunks[10])
    )
    return msg

def gga_parser(msg):
    """
    $GPGGA,234805.000,3906.7106,N,12120.3144,W,1,04,1.77,135.6,M,-22.1,M,,*50
    GGA = namedtuple("GGA", "id utc lat lon qual num_sats hdop msl geoid age ref_id")
    """
    chunks = msg[1:-3].split(',')
    id = chunks[0]
    utc = time_t(
        int(chunks[1][:2]),
        int(chunks[1][2:4]),
        float(chunks[1][4:])
    )
    lat = float(chunks[2][:2]) + float(chunks[2][2:]) / 60.0
    if chunks[3] == 'S':
        lat = -lat
    lon = float(chunks[4][:3]) + float(chunks[4][3:]) / 60.0
    if chunks[5] == 'W':
        lon = -lon

    msg = GGA(
        id,
        utc,
        lat, lon,
        int(chunks[6]),
        int(chunks[7]),
        float(chunks[8]),
        float(chunks[9]),
        float(chunks[11]),
        "" if chunks[13] == "" else float(chunks[13]),
        "" if chunks[14] == "" else int(chunks[14]),
    )
    return msg

In [4]:
def get_msgs(num):
    CR = '\r'
    LF = '\n'
    try:
        ports = comports()
        port = None
        for p in ports:
            if "usbmodem" in p.device:
                port = p.device
                break
        print(port)
        ser = Serial(port, 1000000)
    
        msgs = []
        
        for i in range(num):
            buffer = []
            fin = [0,0]
            c = 0
            while (c != '$'):
                c = ser.read(1).decode("utf8")
            fin[0] = c
            buffer.append(c)
            while (fin[1] != CR and fin[0] != LF):
                c = ser.read(1).decode("utf8")
                buffer.append(c)
                fin[1] = fin[0]
                fin[0] = c
            msg = "".join(buffer[:-2]) # remove \CR \LF
    
            ok = valid_msg(msg)
            if ok:
                msgs.append(msg)
    except Exception as e:
        print(e)
    finally:
        ser.close()
    return msgs

In [6]:
msgs = get_msgs(25)
for msg in msgs:
    hdr = msg[3:6]
    gps = None
    if hdr == "GSA":
        gps = gsa_parser(msg)
    elif hdr == "RMC":
        gps = rmc_parser(msg)
    elif hdr == "GGA":
        gps = gga_parser(msg)
    elif hdr == "GSV":
        # gps = gsv_parser(msg)
        pass
    elif hdr == "VTG":
        # gps = vtg_parser(msg)
        pass
    else:
        print(hdr)

    if gps is not None:
        print(gps)
        print("")

/dev/cu.usbmodem14501
GGA(id='GPGGA', utc=time_t(hr=5, min=20, sec=8.0), lat=39.111875, lon=-121.33844166666667, qual=1, num_sats=8, hdop=1.08, msl=56.4, geoid=-22.1, age='', ref_id='')

GSA(id='GPGSA', mode='A', fix='3D', prns=['20', '16', '27', '04', '30', '09', '07', '08', '', '', '', ''], pdop=1.38, hdop=1.08, vdop=0.86)

RMC(id='GPRMC', utc=time_t(hr=5, min=20, sec=8.0), status='A', lat=39.111875, lon=-121.33844166666667, speed=0.5, track=158.51, data=data_t(day=13, month=8, year=23), mag='')

GGA(id='GPGGA', utc=time_t(hr=5, min=20, sec=9.0), lat=39.111871666666666, lon=-121.33844, qual=1, num_sats=8, hdop=1.08, msl=56.4, geoid=-22.1, age='', ref_id='')

GSA(id='GPGSA', mode='A', fix='3D', prns=['20', '16', '27', '04', '30', '09', '07', '08', '', '', '', ''], pdop=1.38, hdop=1.08, vdop=0.86)

RMC(id='GPRMC', utc=time_t(hr=5, min=20, sec=9.0), status='A', lat=39.111871666666666, lon=-121.33844, speed=0.62, track=168.27, data=data_t(day=13, month=8, year=23), mag='')

GGA(id='GPGGA